# Imports

In [1]:
import yaml
import re
import numpy as np

# Metrics
from scipy.spatial.distance import directed_hausdorff
from scipy.spatial.distance import euclidean
from scipy.spatial.transform import Rotation

from fastdtw import fastdtw

# Visualization
import plotly.graph_objects as go

# Data generation

## Build the project

In [2]:
# %cd ../..
# ! colcon build

## Run nodes and save the outputs to .txt files

In [3]:
# ! ros2 topic echo /new_pose > calculated_pose.txt
# ! ros2 topic echo /clean_pose > original_pose.txt

# Extract position sequence

## Change working directory for the Notebook to read the recordings

In [2]:
%cd ..
%cd recordings/

/home/pablo-portatil/GITHUB/TFG/project_ws/src
/home/pablo-portatil/GITHUB/TFG/project_ws/src/recordings


/home/pablo-portatil/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


## Save the 3D paths

In [9]:
# Test code to add the accumulated error
with open("error_rmse.txt", "r") as file:
    content_error = file.read()
    
# Splitting each message by its separator
message_list_error = re.split(",", content_error)

message_list_error = [float(num) for num in message_list_error]

error_val = np.mean(message_list_error)

print("Mean: " + str(error_val))

print("Min: " + str(min(message_list_error)))

print("Max: " + str(max(message_list_error)))

Mean: 0.005613462993181683
Min: 0.0
Max: 0.013203572765271315


In [5]:
# Read the file
with open("original_pose.txt", "r") as file:
    content_original = file.read()
    
# Splitting each message by its separator
message_list_original = re.split("---", content_original)

original_path = []

# Extract the x y and z values from each message and append them to the path
for i in message_list_original:
    
    # Load the message as a yaml
    yaml_format_original = yaml.safe_load(i)
    # Create a tuple with x y and z values
    pose_format_original = [yaml_format_original["pose"]["pose"]["position"]["x"], 
                            yaml_format_original["pose"]["pose"]["position"]["y"], 
                            yaml_format_original["pose"]["pose"]["position"]["z"]]
    # Append the tuple to the path
    original_path.append(pose_format_original)
    
original_path = np.array(original_path)

In [8]:
# Read the file
with open("calculated_pose.txt", "r") as file:
    content_calculated = file.read()
    
# Splitting each message by its separator
message_list_calculated = re.split("---", content_calculated)

calculated_path = []

# Extract the x y and z values from each message and append them to the path
for i in message_list_calculated:
    
    # Load the message as a yaml
    yaml_format_calculated = yaml.safe_load(i)
    # Create a tuple with x y and z values
    pose_format_calculated = [yaml_format_calculated["pose"]["position"]["x"], 
                              yaml_format_calculated["pose"]["position"]["y"], 
                              yaml_format_calculated["pose"]["position"]["z"]]
    # Append the tuple to the path
    calculated_path.append(pose_format_calculated)
    
calculated_path = np.array(calculated_path)

# Metricas y visualización

## Metricas

### Absolut Trayectory Error (ATE)

In [5]:
def align_trajectories(traj_ref, traj_est):

    # Alings the estimate trayectory to the reference using a Sim(3) transformation

    # Centroids calculation
    centroid_ref = np.mean(traj_ref, axis=0)
    centroid_est = np.mean(traj_est, axis=0)
    
    # Center data
    traj_ref_centered = traj_ref - centroid_ref
    traj_est_centered = traj_est - centroid_est

    # Covariance matrix
    H = traj_est_centered.T @ traj_ref_centered

    # SVD decomposition for optimum rotation search
    U, _, Vt = np.linalg.svd(H)
    
    # Rotation matrix
    R = Vt.T @ U.T 

    # Valid rotation matrix check
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # Apply rotation and translation
    traj_est_aligned = (traj_est_centered @ R.T) + centroid_ref

    return traj_est_aligned

In [6]:
def compute_ate(traj_ref, traj_est):
    # Compute Absolut Trayectory error between two paths
    
    # Aling estimated trayectories
    traj_est_aligned = align_trajectories(traj_ref, traj_est)

    # Compute point to point error
    errors = np.linalg.norm(traj_ref - traj_est_aligned, axis=1)

    # Compute metrics
    ate_mean = np.mean(errors)
    ate_std = np.std(errors)

    return ate_mean, ate_std, errors

In [7]:
print(compute_ate(original_path, calculated_path))

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 1058 is different from 986)

### Hausdorff

In [8]:
d1 = directed_hausdorff(original_path, calculated_path)[0]
d2 = directed_hausdorff(calculated_path, original_path)[0]

hausdorff_distance = max(d1, d2)
print("Hausdorff Distance:", hausdorff_distance)

Hausdorff Distance: 7.364936610263617


### Frechet-like Distance (DTW)

In [9]:
distance, _ = fastdtw(calculated_path, original_path, dist=euclidean)
print("Frechet-like Distance (DTW):", distance)

Frechet-like Distance (DTW): 5437.319814958905


## Visualización

In [9]:
# Create 3D scatter lines
fig = go.Figure()

# Adding Original Path to figure
fig.add_trace(go.Scatter3d(
    x=original_path[:,0], y=original_path[:,1], z=original_path[:,2],
    mode='lines+markers', marker=dict(size=4, color='red'),
    name="Original"
))

# Adding Calculated Path to figure
fig.add_trace(go.Scatter3d(
    x=calculated_path[:,0], y=calculated_path[:,1], z=calculated_path[:,2],
    mode='lines+markers', marker=dict(size=4, color='blue'),
    name="Calculated"
))

# Labels & Layout
fig.update_layout(
    title="Interactive 3D Path Visualization",
    scene=dict(
        xaxis_title="X",
        yaxis_title="Y",
        zaxis_title="Z"
    ),
    margin=dict(l=0, r=0, b=0, t=40)
)

# Show the figure
fig.show()